In [50]:
import pandas as pd


In [64]:
file_path = "/Users/emiliodulay/Documents/Programming/Math-42-Final-Project/Problem_D_Great_Lakes.xlsx"
df = pd.read_excel(file_path)
df.shape

(29, 13)

In [71]:
df.head(5)
df.iloc[2,0]

'Reference: https://www.lre.usace.army.mil/Missions/Great-Lakes-Information/Great-Lakes-Information-2/Water-Level-Data/'

In [ ]:
df.columns = df.iloc[5,:] # Subset df to only keep necessary data
df = df.iloc[6:,].reset_index() # reset index of new df
df = df.drop('index', axis = 1) # drop new index column
df.columns.name = 'index' # rename index column to index

In [66]:
df.tail(5)

,Lake Superior - Mean Water Level,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
24,2018,183.65,183.58,183.54,183.47,183.5,183.56,183.65,183.65,183.7,183.77,183.76,183.67
25,2019,183.62,183.58,183.57,183.61,183.77,183.84,183.86,183.86,183.86,183.88,183.81,183.74
26,2020,183.71,183.64,183.57,183.62,183.64,183.7,183.76,183.81,183.79,183.76,183.71,183.63
27,2021,183.54,183.46,183.42,183.48,183.51,183.55,183.56,183.54,183.51,183.48,183.38,183.31
28,2022,183.24,183.17,183.15,183.24,183.44,183.57,183.62,183.65,183.65,183.6,183.6,183.57
